# **All Imports**

In [85]:
#pip install dash

In [86]:
#pip install jupyter_dash

In [87]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

In [88]:
df_titanic=pd.read_csv('TitanicClean.csv')
df_titanic.drop('Unnamed: 0',axis=1,inplace=True)
df_titanic

,Age,Cabin,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Died,Survived
0,22,M,S,7.2500,0,1,3,male,1,1,0
1,38,C,C,71.2833,0,2,1,female,1,0,1
2,26,M,S,7.9250,0,3,3,female,0,0,1
3,35,C,S,53.1000,0,4,1,female,1,0,1
4,35,M,S,8.0500,0,5,3,male,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1304,29,M,S,8.0500,0,1305,3,male,0,1,0
1305,39,C,C,108.9000,0,1306,1,female,0,0,1
1306,38,M,S,7.2500,0,1307,3,male,0,1,0
1307,29,M,S,8.0500,0,1308,3,male,0,1,0


# **Graph Data Prep**

#Survival

In [89]:
d_survived=df_titanic['Died']
d_survived.replace(0,'Alive',inplace=True)
d_survived.replace(1,'Dead',inplace=True)
d_survived=pd.DataFrame(d_survived.value_counts().reset_index())
d_survived.rename(columns={'index':'Survival'},inplace=True)
d_survived['Survival']=d_survived['Survival'].astype('category')
d_survived

,Survival,Died
0,Dead,815
1,Alive,494


#Age Categorizing

In [90]:
d_ageS=df_titanic.groupby('Age')['Survived'].mean().reset_index()
d_ageS=pd.DataFrame(d_ageS)
d_ageS.rename(columns={'Survived':'Survival Rate'},inplace=True)
d_ageS['Survival Rate']=d_ageS['Survival Rate']*100
bin=np.linspace(min(d_ageS['Age']),max(d_ageS['Age']),5)
group=['Child(0-18 Years)','Youth(19-36 Years)','MiddleAge(37-54 Years)','Old(55-)']
d_ageS['Category']=pd.qcut(d_ageS['Age'],labels=group,q=[0,1/5,1/2,9/10,1])
d_ageS=pd.DataFrame(d_ageS)
d_ageS=d_ageS.reset_index(drop=True)
d_ageS.drop('Age',inplace=True,axis=1)
d_ageS=d_ageS.groupby('Category')['Survival Rate'].mean().reset_index()
d_ageS

,Category,Survival Rate
0,Child(0-18 Years),51.825397
1,Youth(19-36 Years),40.402807
2,MiddleAge(37-54 Years),38.079598
3,Old(55-),25.000000


#Ticket Cost Categorizing

In [91]:
d_fareS=df_titanic.groupby('Fare')['Survived'].mean().reset_index()
d_fareS=pd.DataFrame(d_fareS)
d_fareS.rename(columns={'Survived':'Survival Rate'},inplace=True)
d_fareS['Survival Rate']=d_fareS['Survival Rate']*100
group=['Low(33th%)','Mid(33-50th%)','High(50-75th%)','Top End(75-100th%)']
d_fareS['Cost_Range']=pd.qcut(d_fareS['Fare'],labels=group,q=[0,1/3,1/2,3/4,1])
d_fareS=pd.DataFrame(d_fareS)
d_fareS=d_fareS.reset_index(drop=True)
d_fareS=pd.DataFrame(d_fareS.groupby('Cost_Range')['Survival Rate'].mean().reset_index())
d_fareS

,Cost_Range,Survival Rate
0,Low(33th%),25.620583
1,Mid(33-50th%),43.339073
2,High(50-75th%),40.863894
3,Top End(75-100th%),65.053198


#Cabin Categorizing

In [92]:
d_cabin=df_titanic.groupby('Cabin')['Survived'].mean().reset_index()
d_cabin.rename(columns={'Survived':'Survival Rate'},inplace=True)
d_cabin['Survival Rate']=d_cabin['Survival Rate']*100
d_cabin

,Cabin,Survival Rate
0,A,45.454545
1,B,67.692308
2,C,57.446809
3,D,65.217391
4,E,68.292683
5,F,52.380952
6,G,60.000000
7,M,30.966469
8,T,0.000000


#Socio-Economic Condition

In [93]:
dict={1:'Poor',2:'Middle Class',3:'Wealthy'}

In [94]:
d_socioeco=df_titanic.groupby('Pclass')['Survived'].mean().reset_index()
d_socioeco.rename(columns={'Survived':'Survival Rate'},inplace=True)
d_socioeco['Survival Rate']=d_socioeco['Survival Rate']*100
for i,val in enumerate(d_socioeco['Pclass']):
  d_socioeco['Pclass'][i]=dict[i+1]
d_socioeco

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Pclass,Survival Rate
0,Poor,57.585139
1,Middle Class,42.238267
2,Wealthy,26.939351


#Family Size Categorizing

In [95]:
d_family=df_titanic[['Parch','SibSp','Survived']]
d_family['Fsize']=d_family['Parch']+d_family['SibSp']+1
d_family.drop('Parch',inplace=True,axis=1)
d_family.drop('SibSp',inplace=True,axis=1)
d_family=d_family.groupby('Fsize')['Survived'].mean().reset_index()
d_family.rename(columns={'Survived':'Survival Rate'},inplace=True)
d_family['Survival Rate']=d_family['Survival Rate']*100
d_family

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Fsize,Survival Rate
0,1,29.240506
1,2,53.191489
2,3,55.974843
3,4,72.093023
4,5,22.727273
5,6,20.000000
6,7,31.250000
7,8,12.500000
8,11,18.181818


#**DashBoarding**

In [96]:
colors = {
    'background': '#ADD8E6',
    'text': '#111111'
}

In [100]:
app = JupyterDash(__name__)

fig_survival=px.bar(d_survived,x='Survival',y='Died',title="Survivors",color='Survival')
fig_age=px.bar(d_ageS,x='Category',y='Survival Rate',title='Survivor Rate by Age Categorized',color='Category')
fig_ticket=px.bar(d_fareS,x='Cost_Range',y='Survival Rate',title='Survivor Rate by Ticket Cost Categorized',color='Cost_Range')
fig_cabin=px.bar(d_cabin,y='Cabin',x='Survival Rate',title='Survivor Rate by Cabin Categorized',color='Cabin',orientation='h')
fig_socioeco=px.bar(d_socioeco,x='Pclass',y='Survival Rate',title='Survivor Rate by Socio-Economic Condition',color='Pclass')
fig_family=px.bar(d_family,y='Fsize',x='Survival Rate',title='Survivor Rate by Family Size',color='Fsize',orientation='h',color_continuous_scale='oryel')

app.layout=html.Div(style={'backgroundColor': colors['background']},children=[html.H1('Titanic DashBoard',style={'textAlign':'center','color':'#111111','font-size':35}),
                    html.Br(),
                    html.Br(),
                    html.Div([
                    html.Div(dcc.Graph(figure=fig_survival),style={'width':'50%'}),
                    html.Div(dcc.Graph(figure=fig_age),style={'width':'50%'}),],
                    style={'display':'flex'}),
                    html.Div([
                    html.Div(dcc.Graph(figure=fig_ticket),style={'width':'50%'}),
                    html.Div(dcc.Graph(figure=fig_cabin),style={'width':'50%'})
                    ,],
                    style={'display':'flex'}),
                    html.Div([
                    html.Div(dcc.Graph(figure=fig_socioeco),style={'width':'50%'}),
                    html.Div(dcc.Graph(figure=fig_family),style={'width':'50%'})
                    ,],
                    style={'display':'flex'}),
                    
])
        
# TODO: Build and run dash app 
if __name__=='__main__':
  app.run_server(host='127.0.0.1',port='8000',debug=True)



Dash app running on:


<IPython.core.display.Javascript object>